In [234]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import bs4
import pandas as pd

chrome_options = Options()

chrome_options.add_argument("user-data-dir=C:\\Users\\exaylo\\AppData\\Local\\Google\\Chrome\\User Data\\Default")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
chrome_options.add_argument(f"user-agent={user_agent}")
driver = webdriver.Chrome(options=chrome_options)
driver.delete_all_cookies()
driver.get("https://steamdb.info/app/1623730/charts")   #link zmieniany zaleznie od interesujacej nas gry
time.sleep(10)
page = driver.page_source
soup = bs4.BeautifulSoup(page, 'html.parser')
peak_th = soup.find('th', string=lambda t: 'Peak' in t)
th_index = list(peak_th.parent.find_all('th')).index(peak_th) 
xpath = f"(//th[contains(text(), 'Peak')])[{th_index}]"

while True:
    peak_element = driver.find_element(By.XPATH, xpath)
    peak_element.click()
    time.sleep(2)
    soup=bs4.BeautifulSoup(driver.page_source, 'html.parser')
    sort_up = soup.find('th',class_='sort-header sort-up')
    if sort_up:
        print("Znaleziono element 'sort-header sort-up'. Przerywam pętlę.")
        break
tab = soup.find('tbody',class_='tabular-nums').find_all('tr')
data_list=[]

from datetime import datetime, timedelta
for row in tab:
    columns = row.find_all('td')
    
    # Pobierz datę i opcjonalny <span> (jeśli istnieje)
    date_td = columns[0]
    date_span = date_td.find('span')
    
    if date_span:
        # Jeśli istnieje <span>, pobierz tekst <span> i daty
        span_text = date_span.get('aria-label', 'Brak informacji')
        date_text = date_span.text.strip()
    else:
        # Jeśli nie ma <span>, pobierz tylko datę
        span_text = 'Brak informacji'
        date_text = date_td.text.strip()
    try:
        date_parsed = datetime.strptime(date_text, "%B %Y")
    except ValueError:
        # Jeśli data ma inny format, pomiń ten wiersz
        continue

    date = date_text  # Data w formacie Miesiąc i Rok
    players = columns[1].text.strip().replace(',','')  # Liczba graczy
    player_increase = columns[2].text.strip().replace(',','')  # Przyrost graczy
    percentage_increase = columns[3].text.strip()  # Procentowy przyrost
    data_list.append({
        'Data': date_text,
        'Liczba graczy': players,
        'Przyrost graczy': player_increase,
        'Procentowy przyrost': percentage_increase,
        'Wydarzenie z miesiaca': span_text
    })
df = pd.DataFrame(data_list)
df.to_csv('output.csv', index=False)

driver.quit()

AttributeError: 'NoneType' object has no attribute 'parent'